In [1]:
import os
os.environ['GROQ_API_KEY'] = open('api_key.txt', 'r').read()

In [2]:
from search_engine_versions.bm25_abstract_only import SearchEngine as bm25_abstract_only_search_engine
from search_engine_versions.semantic_abstract_only import SearchEngine as semantic_abstract_only_search_engine
from search_engine_versions.hybrid_abstract_only import SearchEngine as hybrid_abstract_only_search_engine
from search_engine_versions.max_sim_per_context_sentences_abstract_only import SearchEngine as max_sim_per_context_sentences_abstract_only_search_engine
from search_engine_versions.max_sim_cross_context_sentences_abstract_only import SearchEngine as max_sim_cross_context_sentences_abstract_only_search_engine
from search_engine_versions.max_sim_per_context_chunks_abstract_only import SearchEngine as max_sim_per_context_chunks_abstract_only_search_engine
from search_engine_versions.max_sim_cross_context_chunks_abstract_only import SearchEngine as max_sim_cross_context_chunks_abstract_only_search_engine
from search_engine_versions.bm25_title_only import SearchEngine as bm25_title_only_search_engine
from search_engine_versions.bm25_abstract_and_title import SearchEngine as bm25_title_and_abstract_search_engine
from search_engine_versions.semantic_abstract_and_title import SearchEngine as semantic_title_and_abstract_search_engine
from search_engine_versions.hybrid_abstract_and_title import SearchEngine as hybrid_title_and_abstract_search_engine
from search_engine_versions.max_sim_coss_context_chunks_abstract_and_title import SearchEngine as max_sim_cross_context_chunks_title_and_abstract_search_engine
from search_engine_versions.max_sim_per_context_chunks_abstract_and_title import SearchEngine as max_sim_per_context_chunks_title_and_abstract_search_engine
from search_engine_versions.max_sim_cross_context_sentences_abstract_and_title import SearchEngine as max_sim_cross_context_sentences_title_and_abstract_search_engine
from search_engine_versions.max_sim_per_context_sentences_abstract_and_title import SearchEngine as max_sim_per_context_sentences_title_and_abstract_search_engine
# ...

from query_evaluator import evaluate_articles_boolean, evaluate_articles_levels
from query_generator import generate_and_save_queries
import numpy as np
import pandas as pd
import json


# For queries gen
# path = './app/data/example.json'
# with open(path, 'r', encoding='utf-8') as f:
#     data = f.readlines()
#     data = [json.loads(line) for line in data]
# generate_and_save_queries(data, num_queries=100, output_path='./app/data/queries.json')

DATA_DIR = "./data/"
DATA_FILES = ["example.json"]
SPLIT_SIZE_LIMIT = 100

search_engines = {}
'''
Alguns dos nossos modelos serão avaliados utilizando: 
    1. Apenas o abstract como informação para o vespa. O título de um dos
        documentos servirá como query
    2. O título e o abstract como informação para o vespa. Vamos utilizar
        queries geradas por LLM.
    3. Apenas o título como informação para o vespa. O título de um dos
        documentos servirá como query. Esse caso só será avaliado com o BM25.
Para facilitar a execução dos experimentos, criamos um dicionário com os 
modelos que serão avaliados em cada caso.
'''
models = {
    'title_query': [
        "bm25_abstract_only",
        "semantic_abstract_only",
        "hybrid_abstract_only",
        "max_sim_per_context_sentences_abstract_only",
        "max_sim_cross_context_sentences_abstract_only",
        "max_sim_per_context_chunks_abstract_only",
        "max_sim_cross_context_chunks_abstract_only",
    ],
    'llm_query': [
        "bm25_title_only",
        "bm25_title_and_abstract",
        "semantic_title_and_abstract",
        "hybrid_title_and_abstract",
        "max_sim_per_context_sentences_title_and_abstract",
        "max_sim_cross_context_sentences_title_and_abstract",
        "max_sim_per_context_chunks_title_and_abstract",
        "max_sim_cross_context_chunks_title_and_abstract",
    ]
}
search_engines_dict = {
        # "bm25_abstract_only": bm25_abstract_only_search_engine,
        # "semantic_abstract_only": semantic_abstract_only_search_engine,
        # "hybrid_abstract_only": hybrid_abstract_only_search_engine,
        # "max_sim_per_context_sentences_abstract_only": max_sim_per_context_sentences_abstract_only_search_engine,
        # "max_sim_cross_context_sentences_abstract_only": max_sim_cross_context_sentences_abstract_only_search_engine,
        # "max_sim_per_context_chunks_abstract_only": max_sim_per_context_chunks_abstract_only_search_engine,
        # "max_sim_cross_context_chunks_abstract_only": max_sim_cross_context_chunks_abstract_only_search_engine,
        "bm25_title_only": bm25_title_only_search_engine,
        "bm25_title_and_abstract": bm25_title_and_abstract_search_engine,
        "semantic_title_and_abstract": semantic_title_and_abstract_search_engine,
        "hybrid_title_and_abstract": hybrid_title_and_abstract_search_engine,
        "max_sim_per_context_sentences_title_and_abstract": max_sim_per_context_sentences_title_and_abstract_search_engine,
        "max_sim_cross_context_sentences_title_and_abstract": max_sim_cross_context_sentences_title_and_abstract_search_engine,
        "max_sim_per_context_chunks_title_and_abstract": max_sim_per_context_chunks_title_and_abstract_search_engine,
        "max_sim_cross_context_chunks_title_and_abstract": max_sim_cross_context_chunks_title_and_abstract_search_engine,
}

from sheets_connection import SaveMetrics

save_metrics = SaveMetrics()


/Users/bruno.fornaro/Documents/FGV/Projetos/ArticLE/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bruno.fornaro/Documents/FGV/Projetos/ArticLE/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def average_precision(relevances):
    relevant = np.where(relevances == 1)[0] + 1  # get positions of relevant docs, +1 because index is zero-based
    if len(relevant) == 0:
        return 0
    precision_at_k = np.cumsum(relevances) / (np.arange(len(relevances)) + 1)
    return np.sum(precision_at_k[relevances == 1]) / len(relevant)

def dcg(relevances):
    k = len(relevances)
    relevances = np.array(relevances, dtype=float)
    if relevances.size:
        return relevances[0] + np.sum(relevances[1:] / np.log2(np.arange(2, relevances.size + 1)))
    return 0

def ndcg(relevances):
    k = len(relevances)
    dcg_val = dcg(relevances)
    ideal_relevances = sorted(relevances, reverse=True)
    idcg_val = dcg(ideal_relevances)
    return dcg_val / idcg_val if idcg_val > 0 else 0

with open(DATA_DIR + "queries.json") as f:
    queries = f.readlines()
    queries = [json.loads(line) for line in queries]


def test_search_engine(SearchEngine, name):
    search_engine = SearchEngine()
    search_engine.feed_json(DATA_DIR, DATA_FILES, SPLIT_SIZE_LIMIT)
    search_engines[name] = search_engine

    aps = {}
    ndcgs_levels = {}
    ndcgs_boolean = {}

    for query_obj in queries[:10]:
        query = query_obj['query']
        search_results = search_engine.search(query, 10)
        search_results.rename(columns={'body': 'abstract'}, inplace=True)

        evaluated_results_boolean = evaluate_articles_boolean(search_results.to_dict(orient="records"), query, verbose=False)
        evaluated_results_boolean = pd.DataFrame(evaluated_results_boolean)
        evaluated_results_boolean['eval'] = pd.to_numeric(evaluated_results_boolean['eval'])
        print(evaluated_results_boolean)

        evaluated_results_levels = evaluate_articles_levels(search_results.to_dict(orient="records"), query, verbose=False)
        evaluated_results_levels = pd.DataFrame(evaluated_results_levels)
        evaluated_results_levels['eval'] = pd.to_numeric(evaluated_results_levels['eval'])
        print(evaluated_results_levels)

        aps[query_obj['id']] = average_precision(evaluated_results_boolean['eval'])
        ndcgs_levels[query_obj['id']] = ndcg(evaluated_results_levels['eval'])
        ndcgs_boolean[query_obj['id']] = ndcg(evaluated_results_boolean['eval'])
    
    metrics = {
        'aps': aps,
        'ndcgs_levels': ndcgs_levels,
        'ndcgs_boolean': ndcgs_boolean
    }
    return metrics

def get_search_engine_results(SearchEngine, name):
    search_engine = SearchEngine()
    search_engine.feed_json(DATA_DIR, DATA_FILES, SPLIT_SIZE_LIMIT)
    search_engines[name] = search_engine

    # Cria o arquivo com o nome da search engine
    with open(f"{DATA_DIR}search_engine_results/{name}.json", "w") as f:
        for query_obj in queries[:20]:
            query = query_obj['query']
            search_results = search_engine.search(query, 10)
            search_results.rename(columns={'body': 'abstract'}, inplace=True)
            # f.write(query)
            # f.write("\n")
            f.write(search_results.to_json(orient="records"))
            f.write("\n")


def evaluate_and_save_metrics_from_json(search_results, name):
    for i, query_obj in enumerate(queries[:20]):
        query = query_obj['query']
        search_result = search_results[i]

        evaluated_results_boolean = evaluate_articles_boolean(search_result, query, verbose=False)
        evaluated_results_boolean = pd.DataFrame(evaluated_results_boolean)
        evaluated_results_boolean['eval'] = pd.to_numeric(evaluated_results_boolean['eval'])
        print(evaluated_results_boolean)

        evaluated_results_levels = evaluate_articles_levels(search_result, query, verbose=False)
        evaluated_results_levels = pd.DataFrame(evaluated_results_levels)
        evaluated_results_levels['eval'] = pd.to_numeric(evaluated_results_levels['eval'])
        print(evaluated_results_levels)

        # Avarage Precision
        range_binary_map_llm = save_metrics.set_metric_of_query_and_model(
            type_of_metric='binary_map_llm',
            query=query_obj['id'],
            model_name=name,
            result=average_precision(evaluated_results_boolean['eval'])
        )
        print(f"Resultado salvo para o modelo {name} e query {query_obj['id']} no local {range_binary_map_llm}")
        # NDCG Boolean
        range_binary_ndcg_llm = save_metrics.set_metric_of_query_and_model(
            type_of_metric='binary_ndcg_llm',
            query=query_obj['id'],
            model_name=name,
            result=ndcg(evaluated_results_boolean['eval'])
        )
        print(f"Resultado salvo para o modelo {name} e query {query_obj['id']} no local {range_binary_ndcg_llm}")
        # NDCG Levels
        range_4_levels_ndcg_llmm = save_metrics.set_metric_of_query_and_model(
            type_of_metric='4_levels_ndcg_llm',
            query=query_obj['id'],
            model_name=name,
            result=ndcg(evaluated_results_levels['eval'])
        )
        print(f"Resultado salvo para o modelo {name} e query {query_obj['id']} no local {range_4_levels_ndcg_llmm}")


In [4]:

for name, _ in search_engines_dict.items():
    try:
        with open(f"{DATA_DIR}{name}.json", "r") as f:
            search_results = f.readlines()
            search_results = [json.loads(line) for line in search_results]
        print('Começando a rodar para: ', name)
        evaluate_and_save_metrics_from_json(search_results, name)
        print('Terminou de rodar para: ', name)
    except Exception as e:
        print(f"Provavelmente você n tem o arquivo: ./data/{name}.json")
        print(e)
        continue


Começando a rodar para:  bm25_title_only


Evaluating articles:   0%|          | 0/10 [00:00<?, ?it/s]

Provavelmente você n tem o arquivo: ./data/bm25_title_only.json
'abstract'
Começando a rodar para:  bm25_title_and_abstract


Evaluating articles:  10%|█         | 1/10 [00:00<00:05,  1.77it/s]

id: 0704.0001


...

id: 0704.0044


Evaluating articles:  80%|████████  | 8/10 [00:33<00:09,  4.58s/it]

id: 0704.0070


Evaluating articles:  80%|████████  | 8/10 [00:37<00:09,  4.68s/it]


KeyboardInterrupt: 